# Import libs

In [136]:
import sys
sys.path.append('..\\..')
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
from matplotlib import pyplot as plt
reload(gdspy)

<module 'gdspy' from 'C:\\Users\\avasi\\anaconda3\\lib\\site-packages\\gdspy\\__init__.py'>

# Chip parameters, grid

In [137]:
### to have 50 Oms impedance with eps=11.75
tl_core = 10.
tl_gap = 10.
tl_ground = 20.

resonator_core = 10
resonator_gap = 10
resonator_ground = 20

pad_offset = 1100
pad_element_offset = 400
qubit_position_offset = 800

coupler_start = pad_offset + pad_element_offset
coupler_delta = 500
coupler_length = 320
num_couplers = 1

jc = 0.5e-6 # uA/um^2

layers_configuration = {
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':3,
    'air bridges':2,
    'air bridge pads':1,
    'air bridge sm pads':4,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':17,
    'bandages': 25
}

sample = creator.Sample('SrTiO3-resonators-3',layers_configuration,epsilon=15000)


#specify sample vertical and horizontal lengths
sample.chip_geometry.sample_vertical_size=9.7e3
sample.chip_geometry.sample_horizontal_size=9.7e3
central_line_y = sample.chip_geometry.sample_vertical_size/2
chip_edge_ground = elements.ChipEdgeGround(sample.chip_geometry, sample.layer_configuration, sample.pads, 800)
sample.add(chip_edge_ground)

grid_ground = elements.GridGround(sample.chip_geometry, sample.layer_configuration, sample.objects, 
                                  width=4, period=50)
sample.add(grid_ground)

# Pads

In [138]:
pads_left = []
pads_right = []
for pad_side_id in range(3):
    if pad_side_id != 1:
        pad = elements.Pad('pad-left-' + str(pad_side_id),
                           (pad_offset, sample.chip_geometry.sample_vertical_size / 4 * (pad_side_id + 1)), np.pi, tl_core,
                           tl_gap, tl_ground,
                           layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                           **elements.default_pad_geometry())
        pads_left.append(pad)
        sample.add(pad)
        pad = elements.Pad('pad-right-' + str(pad_side_id),
                           (sample.chip_geometry.sample_horizontal_size - pad_offset,
                            sample.chip_geometry.sample_vertical_size / 4 * (pad_side_id + 1)), 0, tl_core,
                           tl_gap, tl_ground,
                           layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                           **elements.default_pad_geometry())
        pads_right.append(pad)
        sample.add(pad)
    
pads_top = []
pads_bottom = []
for pad_side_id in range(3):
    if pad_side_id != 1:
        pad = elements.Pad('pad-bottom-' + str(pad_side_id),
                           (sample.chip_geometry.sample_horizontal_size / 4 * (pad_side_id + 1), pad_offset),
                           -np.pi / 2, tl_core, tl_gap, tl_ground,
                           layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                           **elements.default_pad_geometry())
        pads_bottom.append(pad)
        sample.add(pad)
        pad = elements.Pad('pad-top-' + str(pad_side_id),
                           (sample.chip_geometry.sample_horizontal_size / 4 * (pad_side_id + 1),
                            sample.chip_geometry.sample_vertical_size - pad_offset),
                           np.pi / 2, tl_core, tl_gap, tl_ground,
                           layer_configuration=sample.layer_configuration, chip_geometry=sample.chip_geometry,
                           **elements.default_pad_geometry())
        pads_top.append(pad)
        sample.add(pad)

    
pad0 = pads_left[0]
pad1 = pads_right[0]
pad2 = pads_top[0]
pad3 = pads_bottom[0]
pad4 = pads_top[1]
pad5 = pads_bottom[1]
pad6 = pads_left[1]
pad7 = pads_right[1]

# Resonators

In [139]:
#upper left corner
ul = (sample.chip_geometry.sample_horizontal_size / 4 - 800,
      3*sample.chip_geometry.sample_vertical_size / 4)
ul1 = (sample.chip_geometry.sample_horizontal_size / 4,
      3*sample.chip_geometry.sample_vertical_size / 4 + 800)

ul2 = (sample.chip_geometry.sample_horizontal_size / 4 + 470,
      3*sample.chip_geometry.sample_vertical_size / 4)

ul3 = (sample.chip_geometry.sample_horizontal_size / 4,
      3*sample.chip_geometry.sample_vertical_size / 4 - 470)

sample.connect_cpw(pad6, pad2, port1='port', port2='port', name='right TL', points=[ul, ul3, ul2, ul1])

[CPW "right TL", l=3502.332]

In [140]:
#upper right corner
ur = (3 * sample.chip_geometry.sample_horizontal_size / 4 + 800,
      3 * sample.chip_geometry.sample_vertical_size / 4)

ur1 = (3 * sample.chip_geometry.sample_horizontal_size / 4,
      3 * sample.chip_geometry.sample_vertical_size / 4 + 800)

ur2 = (3 * sample.chip_geometry.sample_horizontal_size / 4,
      3 * sample.chip_geometry.sample_vertical_size / 4 - 230)

ur3 = (3 * sample.chip_geometry.sample_horizontal_size / 4 - 230,
      3 * sample.chip_geometry.sample_vertical_size / 4)



sample.connect_cpw(pad4, pad7, port1='port', port2='port', name='right TL', points=[ur1, ur3, ur2, ur])

[CPW "right TL", l=3007.735]

In [141]:
#lower left corner
ll = (sample.chip_geometry.sample_horizontal_size / 4 - 800, 
       sample.chip_geometry.sample_vertical_size / 4)
ll1 = (sample.chip_geometry.sample_horizontal_size / 4, 
       sample.chip_geometry.sample_vertical_size / 4 - 800)
ll2 = (sample.chip_geometry.sample_horizontal_size / 4 + 640, 
       sample.chip_geometry.sample_vertical_size / 4 + 300)
ll3 = (sample.chip_geometry.sample_horizontal_size / 4 + 300, 
       sample.chip_geometry.sample_vertical_size / 4 + 640)


sample.connect_cpw(pad0, pad3, port1='port', port2='port', name='right TL', points=[ll, ll3, ll2, ll1])

[CPW "right TL", l=4008.854]

In [142]:
#lower right corner
lr = (3 * sample.chip_geometry.sample_horizontal_size / 4 + 230,
      sample.chip_geometry.sample_vertical_size / 4)
lr1 = (3 * sample.chip_geometry.sample_horizontal_size / 4,
      sample.chip_geometry.sample_vertical_size / 4 - 230)


sample.connect_cpw(pad1, pad5, port1='port', port2='port', name='right TL', points=[lr, lr1])

[CPW "right TL", l=2503.221]

In [143]:
# sample.connect_cpw(pad0, pad1, port1='port', port2='port', name='right TL', points=[])

In [144]:
# top1 = (sample.chip_geometry.sample_horizontal_size / 4, 
#         sample.chip_geometry.sample_vertical_size - pad_offset - 2635)

# top2 = (sample.chip_geometry.sample_horizontal_size / 4 * 3, 
#         sample.chip_geometry.sample_vertical_size - pad_offset - 2635)

# sample.connect_cpw(pad2, pad4, port1='port', port2='port', name='right TL', points=[top1, top2])

In [145]:
# bot1 = (sample.chip_geometry.sample_horizontal_size / 4, 
#         pad_offset + 385)

# bot2 = (sample.chip_geometry.sample_horizontal_size / 4 * 3, 
#         pad_offset + 385)

# sample.connect_cpw(pad3, pad5, port1='port', port2='port', name='right TL', points=[bot1, bot2])

In [146]:
sample.draw_design()
sample.watch()

In [147]:
sample.write_to_gds()

Gds file has been writen here:  C:\Users\avasi\Documents\GitHub\QCreator\designs\SrTiO3_resonators/SrTiO3-resonators-3.gds


# Frequencies

In [ ]:
sys, connections, elements_ = sample.get_tls(cutoff=2e9)
f, delta, modes = sys.get_modes()
num_modes=18


In [ ]:
f[:num_modes]/(2e6*np.pi), delta[:num_modes]/(2e6*np.pi), f/(2*np.pi*2*delta)

# S21

In [ ]:
f_ = np.linspace(0.01e9, 1e9, 1601)
# s11 = sample.get_s21('p1', 'p1', f_)
s21 = sample.get_s21('pad-left-0', 'pad-right-0', f_)
s212 = sample.get_s21('pad-top-0', 'pad-top-2', f_)
s213 = sample.get_s21('pad-bottom-0', 'pad-bottom-2', f_)
#plt.plot(f_, np.angle(s21))
#plt.plot(f_, np.angle(s212))
#plt.plot(f_, np.angle(s213))
plt.plot(f_, s21)
plt.plot(f_, s212)
plt.plot(f_, s213)